# Paper Reproduction: "ALBERTA’S FISCAL RESPONSES TO FLUCTUATIONS IN NON-RENEWABLE-RESOURCE REVENUE" in python

## Introduction

This is my replication of the empirical results, tables, and figures produced in a paper by Dr. Ergete Ferede, published by the University of Calgary school of public policy in Volume 11:24, September 2018.

The original paper is here: [https://www.policyschool.ca/wp-content/uploads/2018/09/NRR-Ferede.pdf](https://www.policyschool.ca/wp-content/uploads/2018/09/NRR-Ferede.pdf)

I chose this paper to reproduce for two reasons. The first is pragmatic; the data it uses is all publicly available, so I actually can. The second is that it describes a topic of importance in the province of Alberta, where I live.

You can read the details of what the paper sets out to show in the paper itself, but in brief the idea is to show that provincial government spending increases in the year following an increase in non-renewable resource revenue, but it does not decrease accordingly in the year following declines in the same revenue source. This has a ratcheting effect on public finance that is a contributor to the "royalty rollercoaster" that is Alberta's public finance.

In the following sections I'll go through the code necessary to extract and transform the data set used in the paper, as well as reproduce its key empirical results. Since most economists don't use python, and they make up a key part of my intended audience for this, I'll be adding comments to my code that explicitly describe what some of the functions and methods I'm calling do.

I'm including all of the code necessary to produce this reproduction, since that's a big part of why I'm doing this exercise, but if you're just interested in seeing how my reproduced results compare to the original paper you can skip all the code blocks.

## Setup and data acquisition

This section of the code loads required modules, downloads the required data sets, and reads them into DataFrames.

In [ ]:
from pathlib import Path
import datetime as dt
import requests
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import stats_can
import altair as alt
import seaborn as sns
from arch.unitroot import DFGLS, ADF, PhillipsPerron
from statsmodels.tsa.api import VAR
%matplotlib inline

We start by loading the required libraries that will be used to support the analysis. For reference here are links to the libraries that are being used:

* [Pathlib](https://docs.python.org/3/library/pathlib.html)
* [datetime](https://docs.python.org/3/library/datetime.html)
* [requests](https://requests.readthedocs.io/en/master/)
* [pandas](https://pandas.pydata.org/)
* [pandas_datareader](https://pandas-datareader.readthedocs.io/en/latest/)
* [numpy](https://numpy.org/)
* [stats_can](https://stats-can.readthedocs.io/en/latest/)
* [altair](https://altair-viz.github.io/)
* [seaborn](https://seaborn.pydata.org/)
* [arch](https://arch.readthedocs.io/en/latest/index.html)
* [statsmodels](https://www.statsmodels.org/stable/index.html)
* [matplotlib](https://matplotlib.org/stable/index.html)

### Historical budget data

Functions in this section are concerned with acquiring historical Alberta budget data and reading it into a DataFrame

In [ ]:
def download_budget_data() -> Path:
    """Download the excel file for the analysis from the policy school page.

    Note the readme sheet on the first file. Credit to Kneebone and Wilkins for
    assembling it, and policy school for hosting it.
    
    Originally used this URL, but found it was missing some later heritage
    contributions. After discussion with Dr. Kneebone an updated set has been provided
    https://www.policyschool.ca/wp-content/uploads/2019/01/Provincial-Government-Budget-Data-January-2019FINAL-USE.xlsx

    Returns
    -------
    pathlib.Path
        A path object with the location and name of the data
    """
    print('Downloading data set')

    url = 'https://www.policyschool.ca/wp-content/uploads/2019/03/Provincial-Government-Budget-Data-March-2019.xlsx'
    # send a request to the url for the file
    response = requests.get(
        url,
        stream=True,
        headers={'user-agent': None}
    )
    # create a path object for the file in the data folder above
    # where this notebook is saved with the file named
    # budget.xlsx for easy later access.
    fname = Path('.').joinpath('data').joinpath('budgets.xlsx')
    # write the response from the request to the file in the path specified above
    with open (fname, 'wb') as outfile:
        for chunk in response.iter_content(chunk_size=512):
            if chunk: # filter out keep-alive new chunks
                outfile.write(chunk)
    # Return the location of the file so we can load it later easily
    return fname


def get_budget_file(force_update: bool=False) -> Path:
    """Get the budget file, downloading if required.

    Parameters
    ----------
    force_update: bool
        Download the data file even if you already have it

    Returns
    -------
    pathlib.Path
        A path object with the location and name of the data
    """
    # This is where we're expecting the file to be saved if it exists
    fname = Path('.').joinpath('data').joinpath('budgets.xlsx')
    if not fname.exists() or force_update:
        download_budget_data()
    return fname


def get_date_index(df: pd.DataFrame) -> pd.DatetimeIndex:
    """Helper function to turn budget year strings into datetimes.

    The Fiscal year columns span across years, e.g. 1965-66. In order
    to use all the date indexed functionality I want to convert them into
    an actual datetime format. This function accomplishes that
    
    Parameters
    ----------
    df: pd.DataFrame
        The budget dataframe with the fiscal year style columns
    
    Returns
    -------
    pd.DatetimeIndex
        A datetime index showing January 1 of the beginning of each
        fiscal year for each period.    
    """
    date_index = pd.to_datetime(
        df
        .assign(year=lambda df: df['budget_yr'].str[0:4].astype(int))
        .assign(month=1)
        .assign(day=1)
        [['year', 'month', 'day']]
    )
    return date_index


def read_ab_budget() -> pd.DataFrame:
    """Read Alberta budget data.

    Downloads the data if necessary, reads it in and gives
    the variables easier to work with names

    Returns
    -------
    pd.DataFrame
        Alberta's revenue and expenditure tables
    """
    # Get the budget file, download if necessary  using functions
    # defined above
    fname = get_budget_file()
    df = (
        pd.read_excel(
            fname,
            sheet_name='Alberta',
            # column titles are spaced over 3 rows
            header=3,
            # first column of data is B
            index_col=1,
            # there's a big footnote at the bottom we want to skip
            skipfooter=21
        )
        # Because of the merged cells we get an empty first row
        .loc[lambda x: x.index.notnull()]
        # Not sure where the empty first column comes from but drop it
        .drop(columns='Unnamed: 0')
        .reset_index()
        .rename(columns={
            'index': 'budget_yr',
            'Personal Income Tax': 'personal_income_tax',
            'Corporation Income Tax': 'corporate_income_tax',
            'Retail Sales Tax': 'retail_sales_tax',
            'Federal Cash Transfers': 'federal_cash_transfers',
            'Natural Resource Revenue': 'natural_resource_revenue',
            'Other Own-Source Revenue': 'other_own_source_revenue',
            'Total Revenue': 'total_revenue',
            'Health': 'health_exp',
            'Social Services': 'social_services_exp',
            'Education': 'education_exp',
            'Other Program Expenditures': 'other_program_exp',
            'Total Program Expenditures': 'total_prog_exp',
            'Debt Service': 'debt_service',
            'Total  Expenditures': 'total_exp',
            'Unnamed: 16': 'annual_deficit'
        })
        # Turn the fiscal year string into a datetime object
        .assign(budget_dt=lambda df: get_date_index(df))
        .set_index('budget_dt')
    )
    return df


def read_heritage() -> pd.DataFrame:
    """Read deposits to the heritage trust fund from a separate table.

    The paper nets out contributions to the heritage trust fund when they are
    made, so we have to read them in to be able to net them out of resource revenue.

    They're stored in the same sheet of the workbook, just down below the big table we
    read in with the function above.
    """
    fname = get_budget_file()
    df = (
        pd.read_excel(
            fname,
            sheet_name='Alberta',
            # Have to manually specify column names because of
            # how the table is laid out
            header=None,
            usecols='D:G',
            names=['budget_yr', 'resource_allocation', 'deposits', 'advance_edu'],
            skiprows=71,
            skipfooter=1
        )
        # more fiddly cleaning because of how the table is set up
        # there's a blank row between 1986-87 and when
        # contributions resume in 2005-06
        .loc[lambda df: ~df['budget_yr'].isna()]
        .set_index('budget_yr')
        # missing entries have 0 contributions for that
        # category in that year
        .fillna(0)
        # The three columns are all counted the same
        # for the purposes of this analysis, they just have
        # different labels/classifications depending on the year
        .assign(total_heritage=lambda df: df.sum(axis='columns'))
        # Add a dummy variable to indicate heritage fund deposit years
        .assign(heritage_dummy=1)
        .reset_index()
        # convert the fiscal year column to a datetime index
        .assign(budget_dt=lambda df: get_date_index(df))
        .drop(columns='budget_yr')
        .set_index('budget_dt')
    )
    return df


def clean_budget() -> pd.DataFrame:
    """Combine base budget with heritage deposits.
    
    Pull all the logic together to create one dataframe with all the
    fiscal data for the period of interest.
    
    Returns
    -------
    pd.DataFrame
        The full nominal budget data set.
    """
    budg = read_ab_budget()
    heritage = read_heritage()
    budg_clean = (
        # Start with the budget dataframe
        budg
        # consolidate some revenue categories
        .assign(other_revenue=lambda df: df[['retail_sales_tax', 'federal_cash_transfers', 'other_own_source_revenue']].sum(axis='columns'))
        # Just keep the columns we still need
        .reindex(columns=['personal_income_tax', 'corporate_income_tax', 'natural_resource_revenue', 'other_revenue', 'total_prog_exp', 'debt_service'])
        # add in the heritage contributions data
        .merge(heritage[['total_heritage', 'heritage_dummy']], how='left', left_index=True, right_index=True)
        # Set contributions and the heritage dummy to 0 for years where there were no contributions
        .fillna(0)
        # Net out heritage contributions from natural resources revenue
        .assign(natural_resource_revenue_before_heritage=lambda df: df['natural_resource_revenue'])
        .assign(natural_resource_revenue=lambda df: df['natural_resource_revenue'] - df['total_heritage'])
        # consolidate revenue
        .assign(total_revenue=lambda df: df[['personal_income_tax', 'corporate_income_tax', 'natural_resource_revenue', 'other_revenue']].sum(axis='columns'))
        # consolidate expenditure
        .assign(total_expenditure=lambda df: df[['total_prog_exp', 'debt_service']].sum(axis='columns'))
        # calculate the deficit
        .assign(deficit=lambda df: df['total_expenditure'] - df['total_revenue'])
        # make all the budget numbers floating point
        .astype('float64')
    )
    return budg_clean

### Real Per Capita budget

All of the analysis in the paper is done in terms of real per-capita data. Functions in this section transform the nominal total budget numbers acquired in the previous section into real per-capita figures.

In [ ]:
def periodic_to_budget_annual(df: pd.DataFrame, index_name: str, year_periods: int = 4) -> pd.DataFrame:
    """Take a monthly or quarterly indexed dataframe and annualize it by budget period.

    The inflation and population data we need to convert the budget into
    real per-capita figures are monthly series. We need to get the average
    population and price level for each fiscal year in the data set.

    Rolling mean indexed on January year N+1 is the March to March
    average population for fiscal year N
    Applying a date offset of -1 year and taking only
    January data of these rolling means gives us an average on the
    same basis as the budget dates.

    Parameters
    ----------
    df: pandas.DataFrame
        DataFrame to be piped into this function
    index_name: str
        The name of the date index
    year_periods: int, default 4
        4 for quarterly data (population), 12 for monthly (inflation)

    Returns
    -------
    pd.DataFrame
        An annualized dataframe on a fiscal year basis for comparison
        to annual budget figures.
    """
    df = (
        df
        .copy()
        .rolling(year_periods, closed='left')
        .mean()
        .reset_index()
        .assign(budget_dt=lambda df: df[index_name] - pd.DateOffset(years=1))
        .loc[lambda x: x['budget_dt'].dt.year >= 1965]
        .loc[lambda x: x['budget_dt'].dt.month == 1]
        .drop(columns=index_name)
        .set_index('budget_dt')
        .copy()
    )
    return df


def per_capita_data() -> pd.DataFrame:
    """Read in population data to calculate per capita estimates.

    Quarterly population estimates for Alberta from Statistics Canada
    
    Returns
    -------
    pd.DataFrame
        Fiscal year annualized population estimates for Alberta over the
        reference period.
    """
    table = '17-10-0009-01'
    df = (
        stats_can.table_to_df(table, path='data')
        .loc[lambda x: x['GEO'] == 'Alberta']
        .loc[lambda x: x['REF_DATE'] >= '1965']
        .set_index('REF_DATE')
        [['VALUE']]
        .rename(columns={'VALUE' : 'population'})
        .pipe(periodic_to_budget_annual, 'REF_DATE', 4)
    )
    return df


def inflation_data() -> pd.DataFrame:
    """Read in inflation data to calculate real dollar estimates.

    The whole series is scaled so 2017 budget year is = 1
    
    Returns
    -------
    pd.DataFrame
        Fiscal year annualized inflation data for Alberta over
        the reference period. Normalized to 2017 = 1
    """
    # Alberta inflation doesn't go back far enough, use Canada for earlier dates
    vecs = ('v41692327', 'v41690973')
    df = (
        stats_can.vectors_to_df_local(vecs, path='data', start_date=dt.date(1965, 1, 1))
        .rename(columns={'v41692327': 'ab_inflation', 'v41690973': 'ca_inflation'})
    )
    # fill in with Canadian inflation data where (early) Alberta inflation data is missing.
    mask = df['ab_inflation'].isna()
    # Could probably do some interpolation or scaling before this, but I looked
    # at the raw series and they were pretty comparable
    df.loc[mask, 'ab_inflation'] = df.loc[mask, 'ca_inflation']
    df = (
        df
        .drop(columns='ca_inflation')
        .pipe(periodic_to_budget_annual, 'REF_DATE', 12)
    )
    # Rescale to 2017 = 100 (this is fiscal year 2017,
    # original may have done calendar year)
    inf_2017 = float(df.loc['2017', 'ab_inflation'])
    df = df / inf_2017
    return df


def budget_real_per_capita() -> pd.DataFrame:
    """Get budget data in real per-capita terms.

    Returns
    -------
    pd.DataFrame
        Budget data in real per-capita terms.
    """
    # Read in budget data using the function defined in the 
    # previous section
    clean_budget_df = clean_budget()
    # Everything except the dummy variable gets turned into
    # real per-capita terms
    scale_cols = clean_budget_df.columns.drop('heritage_dummy').tolist()
    # Get population
    per_capita = per_capita_data()
    # Get inflation
    inflation = inflation_data()
    # Combine the datasets, can just use assign because they all
    # have a datetime index
    dfpc = (
        clean_budget_df
        .assign(pop=per_capita)
        .assign(cpi=inflation)
    )
    # rescale to real per capita
    dfpc[scale_cols] = (
        dfpc[scale_cols]
        # original data was in millions of dollars
        .mul(1_000_000)
        # divide by population and inflation for
        # real per-capita
        .div(dfpc['pop'], axis='index')
        .div(dfpc['cpi'], axis='index')
    )
    return dfpc

### Exogenous factors

The paper lists the Alberta employment rate, the Alberta unemployment rate, and the CAD/USD exchange rate as exogenous factors included in the model. Functions in this section acquire that data. I had to do some fiddling to get long enough historical series for some of the factors as you'll note in the code. It's hard to say for sure how the original author sourced this data. I'll just have to compare my tables and charts to his to see if I got close enough.

In [ ]:
def download_historical_cad_usd() -> pd.DataFrame:
    """Get exchange rates from before 1971.

    FRED live data only goes back to 1971, I need a longer series
    This was what I could find. It's annual only, so I can't do it on a budget
    year basis, but hopefully it will be close enough

    This whole function is just some gross munging to read in a table from a web page.
    Once it's called we save it to the data folder so I don't have to re-call it every
    time I run this notebook.
    """
    url = 'https://fxtop.com/en/historical-exchange-rates.php?YA=1&C1=USD&C2=CAD&A=1&YYYY1=1953&MM1=01&DD1=01&YYYY2=2019&MM2=04&DD2=01&LANG=en'
    df = pd.read_html(url)[29]
    headers = df.iloc[0]
    new_df  = (
        pd.DataFrame(df.values[1:], columns=headers)
        .rename(columns={'Year': 'year', 'Average USD/CAD': 'EXCAUS'})
        .assign(month=1)
        .assign(day=1)
        .assign(budget_dt=lambda df: pd.to_datetime(df[['year', 'month', 'day']]))
        .set_index('budget_dt')
        .reindex(columns=['EXCAUS'])
    )
    new_df.to_csv('./data/early_cad_usd.csv')
    return new_df


def read_historical_cad_usd(force_update: bool = False) -> pd.DataFrame:
    """Get exchange rates before 1971.

    This wraps the above function to read in the downloaded data
    if it's available and download and then read it if required.

    Parameters
    ----------
    force_update: bool
        Download the data set even if you already have it

    Returns
    -------
    pd.DataFrame
        Exchange rates from 1965 to 1971
    """
    fname = Path('.').joinpath('data').joinpath('early_cad_usd.csv')
    if not fname.exists() or force_update:
        return download_historical_cad_usd()
    else:
        return pd.read_csv(fname).set_index('budget_dt')


def download_cad_usd() -> pd.DataFrame:
    """Download monthly exchange data from FRED.

    For most of the period of interest I can get monthly
    data from FRED, so I'll do that where possible.

    Returns
    -------
    pd.DataFrame
        Most of the CAD/USD exchange data I need for this analysis.
    """
    df = pdr.get_data_fred('EXCAUS', start=dt.date(1970, 1, 1))
    df.to_csv('./data/cad_usd.csv')
    return df


def read_cad_usd(force_update=False):
    """Get monthly exchange data from FRED.

    This wraps the above function to read in the downloaded data
    if it's available and download and then read it if required.

    Parameters
    ----------
    force_update: bool
        Download the data set even if you already have it

    Returns
    -------
    pd.DataFrame
        Exchange rate data
    """
    fname = Path('.').joinpath('data').joinpath('cad_usd.csv')
    if not fname.exists() or force_update:
        return download_cad_usd()
    else:
        return pd.read_csv(fname, parse_dates=['DATE']).set_index('DATE')


def annual_cad_usd() -> pd.DataFrame:
    """Full series of CAD/USD in fiscal year format.

    Get FRED data and turn the monthly values into annualized on a budget
    basis for as much as possible. Fill in the remainder with calendar annual
    data from fxtop

    Returns
    -------
    pd.DataFrame
        Exchange data on an annualized basis.
    """
    # Create a datetime index of all the points we need
    annual_date_range = pd.date_range('1964-01-01', '2018-01-01', freq='AS', name='budget_dt')
    # Get the old annual stuff to fill in later
    old_df = read_historical_cad_usd()
    df = (
        # get the monthly series
        read_cad_usd()
        # annualize it
        .pipe(periodic_to_budget_annual, 'DATE', 12)
        # add in all the missing dates we need
        .reindex(annual_date_range)
        # fill those missing dates from the old annual data set.
        .fillna(old_df)
    )
    return df


def stats_can_exog() -> pd.DataFrame:
    """Bring in exogenous StatsCan data. Employment and Unemployment rates.

    Returns
    -------
    pd.DataFrame
        Exogenous data required from StatsCan
    """
    # Vectors for monthly series where available
    ur_vec = "v2064516"
    er_vec = "v2064518"
    annual_date_range = pd.date_range('1964-01-01', '2018-01-01', freq='AS', name='budget_dt')
    # for the earlier periods we only have annual data
    old_df = (
        stats_can.table_to_df('36-10-0345-01', path='data')
        # Get Alberta data only
        .loc[lambda x: x['GEO'] == 'Alberta']
        # Keep only the categories we care about
        .loc[lambda x: x['Economic indicators'].isin(['Population', 'Total employment', 'Unemployment rate'])]
        # pivot so the year is the row and the variables are the columns
        .pivot_table(index='REF_DATE', columns='Economic indicators', values='VALUE')
        .rename(columns={'Unemployment rate': 'unemployment_rate'})
        # calculate the employment rate
        .assign(employment_rate=lambda x: (x['Total employment'] / x['Population']) * 100)
        # drop the population, just used for calculating employment rate
        .reindex(columns=['unemployment_rate', 'employment_rate'])
        .rename_axis('budget_dt', axis='index')
        .rename_axis(None, axis='columns')
    )
    # Get monthly data where available
    df = (
        stats_can.vectors_to_df_local([ur_vec, er_vec], path='data', start_date=dt.date(1964, 1, 1))
        .rename(columns={ur_vec: 'unemployment_rate', er_vec: 'employment_rate'})
        # annualize
        .pipe(periodic_to_budget_annual, 'REF_DATE', 12)
        # get the full range of data we want
        .reindex(annual_date_range)
        # fill in the gaps with the old annual series
        .fillna(old_df)
        # Not ideal but even the annual series doesn't go quite back
        # far enough so we have to backfill the earliest available
        # data point
        .fillna(method='bfill')
    )
    return df


def exogenous_variables() -> pd.DataFrame:
    """Bring in exogenous parameters together.

    From the paper:
    We also include other exogenous variables that are likely to affect
    the province’s budget. It is known that the various components of the
    provincial budget can be influenced by the business cycle. Thus, following
    Buettner and Wildsain (2006), we account for the potential effects of the
    business cycle by including one-period lagged changes in the provincial
    employment and unemployment rates. Another important exogenous factor
    that is often cited in provincial budget documents as being important in
    influencing the provincial government’s oil royalty revenue is the Canadian-U.S.
    dollar exchange rate. For this reason, we control for this factor by
    including one period lagged changes in the Canadian-U.S. dollar exchange rate

    Returns
    -------
    pd.DataFrame
        All the necessary exogenous factors for reproducing the paper.
    """
    cadusd = annual_cad_usd()
    ur_er = stats_can_exog()
    df = pd.concat([cadusd, ur_er], axis='columns')
    return df

## Exploratory Figures

### Figure 1
Page 5 of the report charts Non-renewable Resource Revenue, Total Expenditure, and Total Revenue. All are in per-capita 2017 dollars.
Reproducing this chart will be a good starting check that my data extraction and transformation matches the original author's strategy

In [ ]:
df = budget_real_per_capita()

In [ ]:
chart_df = (
    df
    .loc['1970':'2016', ['natural_resource_revenue', 'total_revenue', 'total_expenditure', 'deficit']]
    .rename(columns={
        'natural_resource_revenue': 'Non-renewable Resource Revenue',
        'total_revenue': 'Total Revenue',
        'total_expenditure': 'Total Expenditure'
    })
    .reset_index()
    .melt(id_vars='budget_dt')
)
chart = (
    alt.Chart(chart_df)
    .mark_line()
    .encode(
        x=alt.X('budget_dt:T', axis=alt.Axis(title=None)),
        y=alt.Y('value:Q', axis=alt.Axis(title='Per capita in 2017 dollars')),
        color=alt.Color('variable:N', legend=alt.Legend(title=None, orient='left'))
    )
    .properties(width=800, height=400)
)
chart

In [ ]:
alt.renderers.enable("jupyterlab")

This graph looks very similar to the chart in the paper, with a notable exception of the 1976/1977 budget year. My chart shows Non-renewable Resource Revenue as slightly negative, whereas the original chart has it largely in line with 1975/1976 and 1977/1978. NRR is negative in my chart because I have netted out contributions to the Alberta Heritage Savings Trust Fund (AHSTF). To the best of my understanding, the original paper does the same, and the consistent values between the two in all other years supports that. Quoting the original paper:

>The part of resource revenue that is saved in the AHSTF is not expected to influence the provincial government’s spending and revenue-raising choices. For this reason, in our analysis, we exclude the part of the resource revenue that is saved in the AHSTF from the non-renewable-resource revenue data. 

For comparison, here is the same chart, but without netting AHSTF contributions from resource revenue, it's still netted out of total revenue, but this is just for comparison:

In [ ]:
chart_df = (
    df
    .loc['1970':'2016', ['natural_resource_revenue_before_heritage', 'total_revenue', 'total_expenditure']]
    .rename(columns={
        'natural_resource_revenue_before_heritage': 'Non-renewable Resource Revenue (before AHSTF contributions)',
        'total_revenue': 'Total Revenue',
        'total_expenditure': 'Total Expenditure'
    })
    .reset_index()
    .melt(id_vars='budget_dt')
)
chart = (
    alt.Chart(chart_df)
    .mark_line()
    .encode(
        x=alt.X('budget_dt:T', axis=alt.Axis(title=None)),
        y=alt.Y('value:Q', axis=alt.Axis(title='Per capita in 2017 dollars')),
        color=alt.Color('variable:N', legend=alt.Legend(title=None, orient='left'))
    )
    .properties(width=800, height=400)
)
chart

1976/1977 more closely matches the original chart in the paper, but the remaining years in the period of mid 70s to mid 80s when there were significant contributions clearly do not match. Going forward all analysis in this reproduction will treat NRR as net of AHSTF contributions.

### Figure 2

Page 6 of the paper produces a scatter plot of Real per capita non-renewable resource revenue on the X axis vs. Real per capita budget balance on the Y, along with a linear trend fit.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
chart_df = (
    df
    .loc['1970':'2016', ['natural_resource_revenue', 'deficit']]
    .rename(columns={
        'natural_resource_revenue': 'Non-renewable Resource Revenue',
        'deficit': 'Deficit'
    })
    .assign(balance=lambda df: df['Deficit'] * -1)
    .rename(columns={'balance': 'Budget Balance'})
    .copy()
)
sns.regplot(x='Non-renewable Resource Revenue', y='Budget Balance', data=chart_df);

With the exception of the outlier previously noted in the time series view of the plot, this representation also looks very similar to what was in the original paper.

## Model Specification and estimation

This section combines the previously specified data extraction with transformations necessary to produce summary statistics, statistical tests, and the VAR model itself.

In [ ]:
def model_df_levels():
    """Combine real per capita budget data to get model data in levels
    
    lag exogenous variables (unemployment and employment rates, CAD/USD exchange)
    """
    budg = budget_real_per_capita()
    exog = exogenous_variables()
    df = (
        pd.concat([budg, exog], axis='columns')
        .rename(columns={'total_prog_exp': 'program_expenditure', 'EXCAUS': 'cad_usd'})
        .assign(ur_lag=lambda df: df['unemployment_rate'].shift(periods=1))
        .assign(er_lag=lambda df: df['employment_rate'].shift(periods=1))
        .assign(cad_usd_lag=lambda df: df['cad_usd'].shift(periods=1))
        .reindex(columns=[
            'program_expenditure', 'debt_service', 'corporate_income_tax',
            'personal_income_tax', 'other_revenue', 'natural_resource_revenue',
            'deficit', 'heritage_dummy', 'ur_lag', 'er_lag', 'cad_usd_lag'
        ])
    )
    return df

In [ ]:
mdfl = model_df_levels()

### Sumary statistics for key variables, 1970-71, 2016-17 in levels

Reproduce the top half of table 1 from the paper

In [ ]:
number = "{:0<4,.1f}"
percent = '{:.1%}'
count = "{:0.0f}"

df = (
    mdfl
    .loc['1970':'2016']
    .copy()
    .drop(columns=['heritage_dummy'])
    .reindex(columns=[
        'natural_resource_revenue', 'corporate_income_tax', 'personal_income_tax',
        'other_revenue', 'debt_service', 'program_expenditure', 'deficit', 'ur_lag',
        'er_lag', 'cad_usd_lag'
    ])
    .describe()
    .T
    .style.format({
        'count': count,
        'mean': number,
        'std': number,
        'min': number,
        '25%': number,
        '50%': number,
        '75%': number,
        'max': number
    })
)
df

All the figures that I can validate against (exogenous variables aren't reported in the paper) are reasonably close. The one noted difference is the previously described outlier in natural resource revenue which leads to my minimum for that variable being significantly lower than in the paper. My guess for the observed discrepancies are differences in calculating population or CPI. It will be interesting to see how sensitive the rest of the model is to these relatively small differences in transformation methodology.

### Sumary statistics for key variables, 1970-71, 2016-17, first difference

Reproduce the bottom half of table 1 from the paper

In [ ]:
def model_df_first_diff(mdfl):
    """Produce the first difference of the level model df"""
    df = (
        mdfl
        .diff()
        .loc['1970':'2016']
        .copy()
        .assign(heritage_dummy=mdfl['heritage_dummy']) # don't want to lag diff this
        .assign(constant=1)
        .assign(zero=0)
        .assign(nrrd=lambda df: df[['natural_resource_revenue', 'zero']].min(axis='columns'))
        .assign(nrri=lambda df: df[['natural_resource_revenue', 'zero']].max(axis='columns'))
        .reindex(columns=[
            'natural_resource_revenue', 'nrri', 'nrrd', 'corporate_income_tax', 'personal_income_tax',
            'other_revenue', 'debt_service', 'program_expenditure', 'deficit', 'ur_lag',
            'er_lag', 'cad_usd_lag', 'heritage_dummy', 'constant'
        ])
    )
    return df

In [ ]:
df = (
    model_df_first_diff(mdfl)
    .drop(columns=['heritage_dummy', 'constant'])
    .describe()
    .T
    .style.format({
        'count': count,
        'mean': number,
        'std': number,
        'min': number,
        '25%': number,
        '50%': number,
        '75%': number,
        'max': number
    })
)
df

The outlier in natural resource revenue really skews the first difference max and min, and increases the standard deviation. Again, it will be interesting to see how this impacts the parameter estimates.

### Raw tables
For the purposes of validation, here are the full tables I used to produce both the summary statistics above, as well as all statistical models and tests below

In [ ]:
mdfl

In [ ]:
model_df_first_diff(mdfl)

### Unit-Root Tests

Table A1 in the paper shows the results of unit root tests for both the level and first differenced variables in the model. This section will reproduce those tables

In [ ]:
level_stationarity_df = (
    mdfl
    .loc['1970':'2016']
    .copy()
    .drop(columns=['heritage_dummy'])
    .reindex(columns=[
        'natural_resource_revenue', 'corporate_income_tax', 'personal_income_tax',
        'other_revenue', 'debt_service', 'program_expenditure', 'deficit', 'ur_lag',
        'er_lag', 'cad_usd_lag'
    ])
)

In [ ]:
first_diff_stationarity_df = (
    model_df_first_diff(mdfl)
    .drop(columns=['heritage_dummy', 'constant'])
)

In [ ]:
def stationarity_tests(df):
    tests_dict = {'ADF': ADF, 'Phillips-Perron': PhillipsPerron, 'DF-GLS': DFGLS} 
    cols = df.columns
    tests_df = pd.DataFrame()
    for test_label, test in tests_dict.items():
        for col in cols:
            if test_label != 'Phillips-Perron':
                col_test = test(df[col].dropna(), method='BIC')
            else:
                col_test = test(df[col].dropna())
            test_val = col_test.stat
            test_p = col_test.pvalue
            test_summary = f'val: {test_val:0.3f}, p: {test_p:.1%}'
            tests_df.loc[col, test_label] = test_summary
    return tests_df

In [ ]:
stationarity_tests(level_stationarity_df)

In [ ]:
stationarity_tests(first_diff_stationarity_df)

Documentation on the test tools I used can be found [here](https://arch.readthedocs.io/en/latest/unitroot/tests.html)

There are some interesting differences. Most notable is that on the levels of the deficit series I reject the null hypothesis of a unit root using all three tests at a significance level < 1%. The paper specifically notes that if the deficit is stationary in levels then a Vector Error Correction model can be applied. As the original author's fails to reject the null he implements a Vector AutoRegression model on the first differenced data. In levels the only other series that I find to be stationary is natural resource revenue. ADF on program expenditure would also reject the null at 5% significance, but would fail to reject it using the other two tests.

Looking at the first differenced series, since that's what the paper ultimately ends up using, I also reject the null hypothesis of a unit root for all variables using all tests at a 1% significant *except* program expenditure and deficit using Augmented Dickey Fuller. Those last two tests differ from what's reported in the paper. 

The paper notes that it uses the Schwarz Information Criterion (SIC) for determining optimal lags in the DF-GLS test. It doesn't specify what it's using in the other two tests. For ADF and DF-GLS I used the Schwarz/Bayesian IC (BIC), [which is just another name for SIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion). Phillips-Perron only uses 1 lag and then Newey-West for a long run variance estimator. I also ran these tests using Akaike IC (AIC) for optimal lags for ADF and DF-GLS, with similar results. I'm not sure what explains the discrepancies in my results with what's reported in the paper

### The Model

Take the first differenced dataframe and produce a VAR model using it

In [ ]:
vec_df = (
    model_df_first_diff(mdfl)
    .drop(columns='natural_resource_revenue')
    .dropna()
)
endog_df = vec_df[[
    'nrri', 'nrrd', 'program_expenditure', 'debt_service', 'corporate_income_tax',
     'personal_income_tax', 'other_revenue'
]]
exog_df = vec_df[['ur_lag', 'er_lag', 'cad_usd_lag', 'heritage_dummy']]
model = VAR(endog=endog_df, exog=exog_df, freq='AS')

In [ ]:
# Fit the model with 2 lags
results = model.fit(2)
summary = results.summary()

In [ ]:
# Don't show exogenous parameters since I have nothing to compare them to
results.params.loc['L1.nrri':]

In [ ]:
def highlight_significance(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: <color>'` where
    color is maroon for 1% significance, 
    red for 5% significance,
    orange for 10%, and black otherwise
    """
    
    if val <= 0.01:
        color = 'maroon'
    elif val <= 0.05:
        color = 'red'
    elif val <= 0.1:
        color = 'orange'
    else:
        color = 'black'
    return f'color: {color}'
    

In [ ]:
results.pvalues.loc['L1.nrri':].style.applymap(highlight_significance).format("{:.2%}")

Well this is weird, my coefficient estimates are significantly different than what's in the original paper. Notably I show an increase in program spending in response to an increase *or* a decrease in resource revenue in the first lag, although neither are statistically significant. This is a pretty stark contrast to the original paper, which has the more intuitive result of program spending increasing after a positive resource revenue shock and decreasing with a negative shock (although only the positive response is statistically significant).

At this point I can't account for the discrepancy. Some of my earlier results differed, due to the noted difference in resource revenue in the one year, as well as minor discrepancies presumably based on me doing slightly different real per capita calculations, but I'm surprised the change is this big.

I'll see what my coefficients imply for the rest of the results, but I imagine they're going to be pretty different going forward.

### Impulse Response Functions

The actual results of the paper involve taking the estimated impulse response functions derived from the VAR model and examining their implications. Let's try and reproduce that now.

In [ ]:
irf = results.irf()

#### Table 3
IMPACTS ON ALBERTA’S BUDGET OF A ONE-DOLLAR INNOVATION IN NON-RENEWABLE-RESOURCE
REVENUE (ASYMMETRIC CASE), 1970/71–2016/17 

In [ ]:
irfs = irf.irfs
irf_stderr = irf.stderr()
params = list(results.params.columns)

In [ ]:
params

In [ ]:
def impulse_response(impulse, response):
    imp_ind = params.index(impulse)
    res_ind = params.index(response)
    ir = irfs[:, res_ind, imp_ind]
    se = irf_stderr[:, res_ind, imp_ind]
    imp_name = response + '_impulse'
    se_name = response + '_se'
    df = pd.DataFrame({imp_name: ir, se_name: se})
    return df.loc[1:3].T

def ir_table(impulse):
    responses = params[2:]
    return pd.concat([impulse_response(impulse, response) for response in responses])
ir_table('nrrd')

#### Figure 3a
The total response of program spending to a one-dollar increase in NRR

In [ ]:
# https://github.com/statsmodels/statsmodels/issues/1265
_ = irf.plot(impulse='nrri', response='program_expenditure')

#### Figure 3b
The total response of program spending to a one-dollar decrease in NRR

In [ ]:
_ = irf.plot(impulse='nrrd', response='program_expenditure')

# TODO
Figure 4a, 4b, 5, 6